In [1]:
# data wrangling
import pandas as pd
import numpy as np

from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor, LassoCV
from sklearn.tree import DecisionTreeRegressor

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# py files
import acquire
import prepare
import explore
import model

In [2]:
df = acquire.acquire_agg_data()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169073 entries, 0 to 169072
Data columns (total 10 columns):
serial_number         169072 non-null object
model                 169073 non-null object
capacity_bytes        169073 non-null int64
max(failure)          169073 non-null int64
max(smart_9_raw)      161975 non-null float64
max(smart_5_raw)      161851 non-null float64
max(smart_187_raw)    104189 non-null float64
max(smart_188_raw)    104179 non-null float64
max(smart_197_raw)    161841 non-null float64
max(smart_198_raw)    161841 non-null float64
dtypes: float64(6), int64(2), object(2)
memory usage: 12.9+ MB


In [4]:
df = prepare.prepare(df)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169073 entries, 0 to 169072
Data columns (total 11 columns):
serial_number                    169072 non-null object
manufacturer                     169073 non-null object
model                            169073 non-null object
capacity_terabytes               169073 non-null float64
failure                          169073 non-null int64
drive_age_in_years               161975 non-null float64
reallocated_sectors_count        161851 non-null float64
reported_uncorrectable_errors    104189 non-null float64
command_timeout                  104179 non-null float64
current_pending_sector_count     161841 non-null float64
uncorrectable_sector_count       161841 non-null float64
dtypes: float64(7), int64(1), object(3)
memory usage: 14.2+ MB


In [6]:
df = prepare.unique(df)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162025 entries, 0 to 169067
Data columns (total 11 columns):
serial_number                    162024 non-null object
manufacturer                     162025 non-null object
model                            162025 non-null object
capacity_terabytes               162025 non-null float64
failure                          162025 non-null int64
drive_age_in_years               161965 non-null float64
reallocated_sectors_count        161841 non-null float64
reported_uncorrectable_errors    104186 non-null float64
command_timeout                  104176 non-null float64
current_pending_sector_count     161831 non-null float64
uncorrectable_sector_count       161831 non-null float64
dtypes: float64(7), int64(1), object(3)
memory usage: 14.8+ MB


In [8]:
df = prepare.treat_nulls(df)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161830 entries, 0 to 169067
Data columns (total 11 columns):
serial_number                    161830 non-null object
manufacturer                     161830 non-null object
model                            161830 non-null object
capacity_terabytes               161830 non-null float64
failure                          161830 non-null int64
drive_age_in_years               161830 non-null float64
reallocated_sectors_count        161830 non-null float64
reported_uncorrectable_errors    161830 non-null float64
command_timeout                  161830 non-null float64
current_pending_sector_count     161830 non-null float64
uncorrectable_sector_count       161830 non-null float64
dtypes: float64(7), int64(1), object(3)
memory usage: 14.8+ MB


In [10]:
df = prepare.treat_nulls(df)

In [11]:
# agg_model = df.groupby(['model']).agg({'failure' : 'sum', 'model' : 'count', 'drive_age_in_years':'mean'})
# agg_model.rename(columns={'failure':'failures', 'model':'total_count'}, inplace=True)

# # add a failure rate column
# agg_model['failure_rate_percent'] = agg_model['failures'] / agg_model['total_count'] * 100
# agg_model.sort_values(by=['failures'], ascending = False)

# Early Failure Column ---> Cutoff = 1.6

In [12]:
df = explore.early_failure(df,1.6)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161830 entries, 0 to 169067
Data columns (total 12 columns):
serial_number                    161830 non-null object
manufacturer                     161830 non-null object
model                            161830 non-null object
capacity_terabytes               161830 non-null float64
failure                          161830 non-null int64
drive_age_in_years               161830 non-null float64
reallocated_sectors_count        161830 non-null float64
reported_uncorrectable_errors    161830 non-null float64
command_timeout                  161830 non-null float64
current_pending_sector_count     161830 non-null float64
uncorrectable_sector_count       161830 non-null float64
early_failure                    161830 non-null int64
dtypes: float64(7), int64(2), object(3)
memory usage: 16.1+ MB


# Keep Only Old & Failed

In [14]:
df = explore.old_or_fail(df)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121902 entries, 0 to 168960
Data columns (total 12 columns):
serial_number                    121902 non-null object
manufacturer                     121902 non-null object
model                            121902 non-null object
capacity_terabytes               121902 non-null float64
failure                          121902 non-null int64
drive_age_in_years               121902 non-null float64
reallocated_sectors_count        121902 non-null float64
reported_uncorrectable_errors    121902 non-null float64
command_timeout                  121902 non-null float64
current_pending_sector_count     121902 non-null float64
uncorrectable_sector_count       121902 non-null float64
early_failure                    121902 non-null int64
dtypes: float64(7), int64(2), object(3)
memory usage: 12.1+ MB


# SMART ---> Bool

In [16]:
df = explore.make_binary_values(df)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121902 entries, 0 to 168960
Data columns (total 17 columns):
serial_number                    121902 non-null object
manufacturer                     121902 non-null object
model                            121902 non-null object
capacity_terabytes               121902 non-null float64
failure                          121902 non-null int64
drive_age_in_years               121902 non-null float64
reallocated_sectors_count        121902 non-null float64
reported_uncorrectable_errors    121902 non-null float64
command_timeout                  121902 non-null float64
current_pending_sector_count     121902 non-null float64
uncorrectable_sector_count       121902 non-null float64
early_failure                    121902 non-null int64
smart_5_nonzero                  121902 non-null bool
smart_187_nonzero                121902 non-null bool
smart_188_nonzero                121902 non-null bool
smart_197_nonzero                121902 non-null b

# Remove Manufacturers

In [18]:
df = explore.remove_manufacturers(df)

# Split Data

In [19]:
def split_my_data(df):
    X = df.drop(columns = ['serial_number', 'model', 'failure', 'early_failure', 'drive_age_in_years', 'reallocated_sectors_count', 'reported_uncorrectable_errors', 'command_timeout', 'current_pending_sector_count', 'uncorrectable_sector_count'])
    y = df[['early_failure']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .80, random_state = 123, stratify=df.early_failure)
    return X, y, X_train, X_test, y_train, y_test

In [20]:
X, y, X_train, X_test, y_train, y_test = split_my_data(df)

In [21]:
len(X_train), len(y_train), len(X_test), len(y_test)

(97520, 97520, 24380, 24380)

In [22]:
X_train.columns

Index(['manufacturer', 'capacity_terabytes', 'smart_5_nonzero',
       'smart_187_nonzero', 'smart_188_nonzero', 'smart_197_nonzero',
       'smart_198_nonzero'],
      dtype='object')

# Encode

In [23]:
#df = model.encode_hot(df)

In [24]:
X_train, X_test = model.encode_hot(X_train, X_test, col_name = 'manufacturer')

In [25]:
X_train = X_train.drop(columns = 'manufacturer')

In [26]:
X_test = X_test.drop(columns = 'manufacturer')